In [14]:
import joblib
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import GRU, LSTM, Dense, SimpleRNN
from tensorflow.keras.models import Sequential

In [15]:
mmn_EGG_time_series = joblib.load("../features/mmnEEGtimeSeries.sav")

In [16]:
target = mmn_EGG_time_series["cateory"].to_numpy()

In [17]:
data = []
for key, value in mmn_EGG_time_series["mmn_eeg"].items():
    data.append(value["1000Hz"])

data = np.array(data)

In [19]:
encoder = LabelEncoder()
labels = encoder.fit_transform(target)
labels

array([1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1])

In [20]:
def build_lstm_model(data, target, test_size=0.2, random_state=42):
    """
    Builds and trains an LSTM model for time series prediction.

    Args:
        data: The input time series data.
        target: The target variable to be predicted.
        test_size: The proportion of data to be used for testing.
        random_state: The random seed for splitting the data.

    Returns:
        A trained LSTM model.
    """

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        data,
        target,
        test_size=test_size,
        random_state=random_state,
    )

    # Standardize the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[2]))
    X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[2]))

    # Define LSTM model
    lstm_model = Sequential()
    lstm_model.add(
        LSTM(
            units=50,
            return_sequences=False,
            input_shape=(X_train.shape[1], X_train.shape[2]),
        )
    )
    lstm_model.add(Dense(units=1, activation="sigmoid"))

    loss_fn = "binary_crossentropy"
    optimizer = "adam"

    # Compile the model
    lstm_model.compile(loss=loss_fn, optimizer=optimizer, metrics=["accuracy"])

    # Add early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor="val_loss", patience=3)

    # Train the model
    lstm_model.fit(
        X_train_scaled.reshape(X_train.shape),
        y_train,
        epochs=10,
        batch_size=8,
        validation_data=(X_test_scaled.reshape(X_test.shape), y_test),
        callbacks=[early_stopping],
    )

    return lstm_model


# Train the model
model = build_lstm_model(data, labels)

# Evaluate the model
model.evaluate(X_test_scaled.reshape(X_test.shape), y_test)

# Make predictions
predictions = model.predict(X_test_scaled.reshape(X_test.shape))

Epoch 1/10
5/5 [==============================] - 6s 437ms/step - loss: 0.6885 - accuracy: 0.5000 - val_loss: 0.7132 - val_accuracy: 0.5556
Epoch 2/10
5/5 [==============================] - 0s 24ms/step - loss: 0.6173 - accuracy: 0.6667 - val_loss: 0.7122 - val_accuracy: 0.5556
Epoch 3/10
5/5 [==============================] - 0s 33ms/step - loss: 0.5621 - accuracy: 0.6944 - val_loss: 0.7005 - val_accuracy: 0.4444
Epoch 4/10
5/5 [==============================] - 0s 30ms/step - loss: 0.5248 - accuracy: 0.8611 - val_loss: 0.6877 - val_accuracy: 0.3333
Epoch 5/10
5/5 [==============================] - 0s 31ms/step - loss: 0.4819 - accuracy: 0.8889 - val_loss: 0.6795 - val_accuracy: 0.3333
Epoch 6/10
5/5 [==============================] - 0s 32ms/step - loss: 0.4418 - accuracy: 0.9167 - val_loss: 0.6685 - val_accuracy: 0.5556
Epoch 7/10
5/5 [==============================] - 0s 29ms/step - loss: 0.4049 - accuracy: 0.9444 - val_loss: 0.6542 - val_accuracy: 0.5556
Epoch 8/10
5/5 [==========

UnimplementedError: Graph execution error:

Detected at node binary_crossentropy/Cast defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\Victor Momodu\Documents\Programming\Machine Learning\Code\AI-Driven-EEG-Schizophrenia-Diagnosis\.venv\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\Victor Momodu\Documents\Programming\Machine Learning\Code\AI-Driven-EEG-Schizophrenia-Diagnosis\.venv\Lib\site-packages\traitlets\config\application.py", line 1077, in launch_instance

  File "c:\Users\Victor Momodu\Documents\Programming\Machine Learning\Code\AI-Driven-EEG-Schizophrenia-Diagnosis\.venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\Victor Momodu\Documents\Programming\Machine Learning\Code\AI-Driven-EEG-Schizophrenia-Diagnosis\.venv\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Python